# Inference for numerical data
# North Carolina births
In 2004, the state of North Carolina released a large data set containing information on births recorded in this state. This data set is useful to researchers studying the relation between habits and practices of expectant mothers and the birth of their children. We will work with a random sample of observations from this data set.

Exploratory analysis
Load the `nc` data set into our workspace.

In [1]:
import pandas as pd
nc = pd.read_csv('nc.csv')